In [ ]:
import pandas as pd
import numpy as np
import cv2

from tqdm import tqdm

from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

In [ ]:
### OLD

# Version1 = "10Fold_CIB" # 10Fold_sigmoid, 10Fold_CIB_V0, 10Fold_CIBLSX
# Version2 = "_V0_M1_avg" # _V0_M1_avg, _V0_M1
# DIR = "C:/Users/brdd/OneDrive - ZHAW/xAI_stroke/AttentionMaps/" + Version1 + Version2[:6]
# oc_gc = "oc" # oc, gc

# # heatmaps= np.load(DIR + "/all_heatmaps_" + Version1 + Version2[3:] + "_" + oc_gc + "_bothcl.npy")
# heatmaps= np.load(DIR + "/all_heatmaps_" + Version1 + Version2[3:] + "_" + oc_gc + "_predcl.npy")

# # meta_dat = pd.read_csv(DIR + "/all_tab_results_" + Version1 + Version2 + ".csv")
# meta_dat = pd.read_csv(DIR + "/all_tab_results_hm_unc_" + Version1 + Version2[3:] + "_gc_bothcl.csv")

# meta_dat = meta_dat.sort_values(by=["p_id"])

In [ ]:
### NEW

# Define version and directory
Version1 = "CIB" # CIB, CIBLSX
Version2 = "_M2_wgt" # _M3_wgt, _M2_wgt
DIR = "C:/Users/brdd/OneDrive - ZHAW/xAI_stroke/AttentionMaps/" + Version1 + Version2[:3]

# chose gradcam or occlusion
gc_oc = "gc"

# Load Data
heatmaps= np.load(DIR + "/all_heatmaps_" + Version1 + Version2 + "_" + gc_oc + "_predcl_unnormalized.npy")
X = np.load("C:/Users/brdd/Documents/x_AI/Data/prepocessed_dicom_3d.npy")

meta_dat = pd.read_csv(DIR + "/all_tab_results_hm_unc_" + Version1 + Version2 + "_" + gc_oc + "_predcl_unnormalized.csv")

meta_dat = meta_dat.sort_values(by=["p_id"])


In [ ]:
heatmaps.max()

In [ ]:
idx = np.where(meta_dat["y_pred_class_avg_w"] == 1)

In [ ]:
hm1 = np.take(heatmaps, idx, axis = 0).squeeze().mean(axis = 0).mean(axis = 2)
plt.imshow(hm1, cmap = "jet", vmin = hm1.min(), vmax = hm1.max())

In [ ]:
heatmaps_norm = []
for i in range(heatmaps.shape[0]):
    if heatmaps[i].max() == heatmaps[i].min():
        hm = np.zeros(heatmaps[i].shape)
    else:
        hm = (heatmaps[i] - heatmaps[i].min()) / (heatmaps[i].max() - heatmaps[i].min())
    heatmaps_norm.append(hm)
heatmaps_norm = np.array(heatmaps_norm)

hm2 = np.take(np.array(heatmaps_norm), idx, axis = 0).squeeze().mean(axis = 0).mean(axis = 2)
plt.imshow(hm2, cmap = "jet", vmin = hm2.min(), vmax = hm2.max())

In [ ]:
heatmaps_sig = []
for i in range(heatmaps.shape[0]):
    p = meta_dat.y_pred_trafo_avg_w[i]
    hm = heatmaps[i] * p * (1-p)
    heatmaps_sig.append(hm)
heatmaps_sig = np.array(heatmaps_sig)

hm3 = np.take(np.array(heatmaps_sig), idx, axis = 0).squeeze().mean(axis = 0).mean(axis = 2)
plt.imshow(hm3, cmap = "jet", vmin = hm3.min(), vmax = hm3.max())

In [ ]:
## take only favorable or unfavorable cases
# heatmaps = heatmaps[meta_dat["unfavorable"] == 0]
# meta_dat = meta_dat[meta_dat["unfavorable"] == 0]

## take only as favorable or as unfavorable predicted cases
# heatmaps = heatmaps[meta_dat["y_pred_class_avg"] == 1]
# meta_dat = meta_dat[meta_dat["y_pred_class_avg"] == 1]

In [ ]:
heatmaps_2d = heatmaps.reshape(heatmaps.shape[0], -1)
heatmaps_mid = heatmaps[:, :, :, 14, :]
heatmaps_avg = np.mean(heatmaps, axis=3)

In [ ]:
print(heatmaps_2d.min(), heatmaps_2d.max())

In [ ]:
from tensorflow import keras

# Load the pre-trained CNN model
model = keras.applications.ResNet50(weights='imagenet', 
                                 input_shape=(128,128,3),
                                 include_top=False)
# VGG16, ResNet50, ResNet101

# Freeze all backbone layers
for layer in model.layers:
    layer.trainable = False

print(model.output)


In [ ]:
def pred_all_layers(heatmap):
    # one dimension
    if heatmap.shape[3] == 1: 
        cnn_features = model.predict(np.repeat(heatmap, 3, -1))

        # average over the output feature maps
        cnn_features = np.mean(cnn_features, axis=(1,2))

    # original dimension
    elif heatmap.shape[3] == 28:
        all_cnn_features = []

        for i in range(28):
            cnn_features = model.predict(np.repeat(heatmap[:, :, :, i, :], 3, -1))

            # average over the output feature maps
            all_cnn_features.append(np.mean(cnn_features, axis=(1,2)))

        cnn_features = np.moveaxis(np.array(all_cnn_features), source=0, destination=-1)

    return cnn_features

In [ ]:
# cnn_features = pred_all_layers(heatmaps_mid)
cnn_features = pred_all_layers(heatmaps_avg)
# cnn_features = pred_all_layers(heatmaps)

In [ ]:
features = cnn_features.reshape(cnn_features.shape[0], -1)

features.shape

In [ ]:
from sklearn.decomposition import PCA

# Perform PCA on features
pca = PCA(n_components=2, random_state=42)
pca_result = pca.fit_transform(features)

# Plot the PCA result
plt.scatter(pca_result[:, 0], pca_result[:, 1], c=meta_dat['unfavorable'], alpha=0.8)
plt.title('PCA Representation of features')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.show()


In [ ]:
print(pca.explained_variance_ratio_)

In [ ]:
# Perform t-SNE on features
tsne = TSNE(n_components=2, random_state=42)
tsne_result = tsne.fit_transform(features)

plt.scatter(tsne_result[:, 0], tsne_result[:, 1], c=meta_dat['unfavorable'], alpha=0.8)
# plt.scatter(tsne_result[:, 0], tsne_result[:, 1], c=meta_dat['y_pred_class'], alpha=0.8)
plt.title('t-SNE Representation of features')
plt.xlabel('Dimension 1')
plt.ylabel('Dimension 2')
plt.show()


In [ ]:
import umap

# Perform UMAP on features
umap_result = umap.UMAP(n_components=2, random_state=42).fit_transform(features)

# Plot the UMAP result
plt.scatter(umap_result[:, 0], umap_result[:, 1], c=meta_dat['unfavorable'], alpha=0.8)
plt.title('UMAP Representation of features')
plt.xlabel('UMAP Dimension 1')
plt.ylabel('UMAP Dimension 2')
plt.show()


In [ ]:
from matplotlib.patches import Rectangle

def min_max_norm(x):
    return (x - np.min(x)) / (np.max(x) - np.min(x))

# Compute the coordinates of the image on the plot
# adapted from https://learnopencv.com/t-sne-for-feature-visualization/
def compute_plot_coordinates(image, x, y):
    image_height, image_width, _ = image.shape
 
    # compute the image center coordinates on the plot
    center_x = x
 
    # in matplotlib, the y axis is directed upward
    # to have the same here, we need to mirror the y coordinate
    center_y = y
 
    # knowing the image center,
    # compute the coordinates of the top left and bottom right corner
    # tl_x = center_x - int(image_width / 2)
    # tl_y = center_y - int(image_height / 2)
    tl_x = center_x
    tl_y = center_y
 
    br_x = tl_x + image_width
    br_y = tl_y + image_height
 
    return tl_x, tl_y, br_x, br_y

# adapted from: https://stackoverflow.com/questions/43261338/adding-a-border-to-and-image-in-my-code
def frame_image(img, frame_width):
    b = frame_width # border size in pixel
    ny, nx = img.shape[0], img.shape[1] # resolution / number of pixels in x and y
    if img.ndim == 3: # rgb or rgba array
        framed_img = np.zeros((b+ny+b, b+nx+b, img.shape[2]))
    elif img.ndim == 2: # grayscale image
        framed_img = np.zeros((b+ny+b, b+nx+b))
    framed_img[b:-b, b:-b] = img
    return framed_img


def plot_image_on_represantion(dim_red_result, maps, meta_info = None,
                               pat_ids = None,
                               yx_ratio = 1,
                               im_size = 128*20, map_col = "jet", 
                               alpha_hm = 0.8, alpha_edge = 0.5, edge_width = 3,
                               style=None,
                               save_name = None):

    dim_red_result[:, 0] = min_max_norm(dim_red_result[:, 0])
    dim_red_result[:, 1] = min_max_norm(dim_red_result[:, 1])

    dim_red_result = (dim_red_result * (im_size - 128)).astype(int)
    if yx_ratio != 1:
        dim_red_result[:, 0] = (dim_red_result[:, 0] * yx_ratio).astype(int)

    # image = (min_max_norm(mid_maps) * 255).astype(int)
    if np.min(maps) < 0:
        image = ((maps+1)/2 * 255).astype(int)
    else:
        image = (maps * 255).astype(int)

    # plot   
    plot_labels = ('true favorable & predicted favorable', 
                   'true favorable & predicted unfavorable ', 
                   'true unfavorable & predicted favorable ', 
                   'true unfavorable & predicted unfavorable')
    plot_colors = ("limegreen", "gold", "magenta", "red")

    plt.style.use(style)

    for i in tqdm(range(dim_red_result.shape[0])):
        im = image[i]
        tl_x, tl_y, br_x, br_y = compute_plot_coordinates(im, dim_red_result[i, 0], dim_red_result[i, 1])

        # plt.imshow(im, extent=(tl_x, br_x, tl_y, br_y), cmap=map_col, alpha = alpha_hm, vmin = (heatmaps[i].min()*255).astype(int), vmax = (heatmaps[i].max()*255).astype(int))
        plt.imshow(im, extent=(tl_x, br_x, tl_y, br_y), cmap=map_col, alpha = alpha_hm, vmin = im.min(), vmax = im.max())
        # plt.imshow(im, extent=(tl_x, br_x, tl_y, br_y), cmap=map_col, alpha = alpha_hm, vmin = image.min(), vmax = image.max())


        if meta_info is not None:
            rect = Rectangle((tl_x, tl_y), im.shape[0], im.shape[1], angle=0.0,
                  edgecolor = plot_colors[meta_info[i]],
                  linewidth=edge_width, facecolor='none', alpha = alpha_edge)
            plt.gca().add_patch(rect)

        if pat_ids is not None:
            plt.text(tl_x, tl_y+(104), pat_ids[i], fontsize=8, color = "black")

    plt.scatter(dim_red_result[:, 0], dim_red_result[:, 1], s=0.01)
    
    plt.legend(plot_labels, labelcolor = plot_colors, handlelength=1, handleheight=1, fontsize = 14)
    
    ax = plt.gca()
    leg = ax.get_legend()
    for (i, element) in enumerate(leg.legend_handles):
        element.set_edgecolor(plot_colors[i])

    plt.tight_layout()
    plt.axis('off')

    if save_name is not None:
        plt.savefig(save_name, dpi=300, bbox_inches='tight')

    plt.show()

Red and Yellow Rectangles are expected to be near each other.
And Green and Pink Rectangles should also be similar.

In [ ]:
plt.figure(figsize=(15, 10))


# fav & fav pred 0 => 0, fav & unfav pred => 1, unfav & fav pred => 2, unfav & unfav pred => 3
meta_info = list(meta_dat["unfavorable"] * 2 + meta_dat["y_pred_class_avg"])
pat_ids = list(meta_dat["p_id"])

plot_image_on_represantion(tsne_result, heatmaps_avg, 
                           meta_info,
                           pat_ids = None, # None, pat_ids
                           yx_ratio = 1.5,
                           im_size = 128*20, map_col = "jet", 
                           alpha_hm = 0.85, alpha_edge = 0.6, 
                           edge_width = 3,
                           style = "default", # dark_background, default
                           save_name = None, # DIR + "/tsne_" + Version1 + Version2 + "_" + gc_oc + "_predcl_unnormalized.png"
)


In [ ]:
# Find the optimal number of clusters using the elbow method
wcss = []
max_clusters = 10  # Maximum number of clusters to consider
for n_clusters in range(1, max_clusters + 1):
    kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init='auto')
    kmeans.fit(features)
    wcss.append(kmeans.inertia_)

# Plot the within-cluster sum of squares (WCSS) against the number of clusters
plt.plot(range(1, max_clusters + 1), wcss)
plt.title('Optimal Number of Clusters')
plt.xlabel('Number of Clusters')
plt.ylabel('Within-Cluster Sum of Squares (WCSS)')
plt.show()
